In [ ]:
# Setup: install Qiskit (runs automatically in Colab, no-op in Binder)
!pip install -q qiskit qiskit-aer qiskit-ibm-runtime pylatexenc

In [ ]:
# Additional dependencies for this notebook
!pip install -q gem-suite

*Dauer: Ungefähr 3 Minuten uffm Heron r2 Prozessor (ACHTUNG: Das is nur ne Schätzung. Deine Laufzeit kann annerscht sein.)*

## Hintergrund

Das Tutorial zeigt, wie mr'n Nishimori Phasenübergang uffm IBM&reg; Quantenprozessor durchführt. Das Experiment wurde ursprünglich beschrieben in [*Realizing the Nishimori transition across the error threshold for constant-depth quantum circuits*](https://arxiv.org/abs/2309.02863).

Der Nishimori Phasenübergang bezieht sich uff den Übergang zwischn kurz- un langstreckengeordneten Phasen im Zufalls-Bindungs-Ising-Modell. Uffm Quantencomputer zeigt sich de langstreckengeordnete Phase als'n Zustand, wo de Qubits übern ganzen Gerät verschrängt sin. Dieser hochgradig verschränkte Zustand wird mitm *generation of entanglement by measurement* (GEM) Protokoll erzeugt. Durchs Verwenden von Messungen innerhalb vom Schaltkreis kann das GEM-Protokoll Qubits übern ganzen Gerät verschränken mit Schaltkreisen von nur konstanter Tiefe. Hier verwenden mr de Implementierung vom GEM-Protokoll ausm [GEM Suite](https://github.com/qiskit-community/gem-suite) Software-Paket.

## Voraussetzungen

Bevor de mitm Tutorial anfängst, sorge dafür, dass de das installiert hast:

- Qiskit SDK v1.0 oder neuer, mit [Visualisierung](https://docs.quantum.ibm.com/api/qiskit/visualization) Unterstützung
- Qiskit Runtime v0.22 oder neuer ( `pip install qiskit-ibm-runtime` )
- GEM Suite ( `pip install gem-suite` )

## Vorbereitung

In [2]:
import matplotlib.pyplot as plt

from collections import defaultdict

from qiskit_ibm_runtime import QiskitRuntimeService

from qiskit.transpiler import generate_preset_pass_manager

from gem_suite import PlaquetteLattice
from gem_suite.experiments import GemExperiment

## Schritt 1: Klassische Eingaben uff'n Quantenproblem abbilden
Das GEM-Protokoll arbeitet uffm Quantenprozessor mit Qubit-Konnektivität, de durchn Gitter beschrieben wird. De heutigen IBM Quantenprozessoren verwenden das [Heavy-Hex-Gitter](https://www.ibm.com/quantum/blog/heavy-hex-lattice). De Qubits vom Prozessor werden in *Plaketten* einjedeilt, je nachdem in welcher Einheitszelle vom Gitter se liegen. Weil'n Qubit in mehr als eener Einheitszelle vorkommen kann, sin de Plaketten nich disjunkt. Uffm Heavy-Hex-Gitter hat ne Plakette 12 Qubits. De Plaketten selbst bilden ooch'n Gitter, wo zwei Plaketten verbunden sin, wennse irgendwelche Qubits teilen. Uffm Heavy-Hex-Gitter teilen benachbarte Plaketten 3 Qubits.

Im GEM Suite Software-Paket is de Grundklasse für de Implementierung vom GEM-Protokoll `PlaquetteLattice`, was das Gitter von de Plaketten darstellt (was annerscht is wie das Heavy-Hex-Gitter). Ne `PlaquetteLattice` kann ausner Qubit Coupling Map initialisiert wern. Momentan wern nur Heavy-Hex Coupling Maps unterstützt.

De nächste Code-Zelle initialisiert'n Plaketten-Gitter ausner Coupling Map vonm IBM Quantenprozessor. Das Plaketten-Gitter umfasst nich immer de ganze Hardware. Zum Beispiel hat `ibm_torino` 133 Qubits insgesamt, aber das größte Plaketten-Gitter, was uffs Gerät passt, braucht nur 125 davon un umfasst insgesamt 18 Plaketten. Ähnliches kann mr ooch bei IBM Quantum&reg; Geräten mit annern Qubit-Zahlen beobachten.

In [ ]:
# QiskitRuntimeService.save_account(channel="ibm_quantum", token="<YOUR_API_KEYN>", overwrite=True, set_as_default=True)
service = QiskitRuntimeService()
backend = service.least_busy(
    operational=True, simulator=False, min_num_qubits=127
)
plaquette_lattice = PlaquetteLattice.from_coupling_map(backend.coupling_map)

print(f"Number of qubits in backend: {backend.num_qubits}")
print(
    f"Number of qubits in plaquette lattice: {len(list(plaquette_lattice.qubits()))}"
)
print(f"Number of plaquettes: {len(list(plaquette_lattice.plaquettes()))}")

Number of qubits in backend: 133
Number of qubits in plaquette lattice: 125
Number of plaquettes: 18


You can visualize the plaquette lattice by generating a diagram of its graph representation. In the diagram, the plaquettes are represented by labeled hexagons, and two plaquettes are connected by an edge if they share qubits.

In [7]:
plaquette_lattice.draw_plaquettes()

<Image src="../docs/images/tutorials/nishimori-phase-transition/extracted-outputs/625882a4-faeb-4d96-b441-c989f43c4dea-0.avif" alt="Output of the previous code cell" />

De kannst das Plaketten-Gitter visualisieren, indem de'n Diagramm von seiner Graph-Darstellung erzeugst. Im Diagramm wern de Plaketten als beschriftete Sechsecke dargestellt, un zwei Plaketten sin durchne Kante verbunden, wennse Qubits teilen.

In [8]:
# Get a list of the plaquettes
plaquettes = list(plaquette_lattice.plaquettes())
# Display information about plaquette 0
plaquettes[0]

PyPlaquette(index=0, qubits=[0, 1, 2, 3, 4, 15, 16, 19, 20, 21, 22, 23], neighbors=[3, 1])

![Output of the previous code cell](../docs/images/tutorials/nishimori-phase-transition/extracted-outputs/625882a4-faeb-4d96-b441-c989f43c4dea-0.avif)

De kannst Informationen über einzelne Plaketten abrufen, wie zum Beispiel de Qubits, de se enthalten, mitder `plaquettes` Methode.

In [9]:
plaquette_lattice.draw_qubits()

<Image src="../docs/images/tutorials/nishimori-phase-transition/extracted-outputs/a19d63ce-3572-4081-a008-c1332fbbe303-0.avif" alt="Output of the previous code cell" />

In addition to the qubit labels and the edges indicating which qubits are connected, the diagram contains three additional pieces of information that are relevant to the GEM protocol:
- Each qubit is either shaded (gray) or unshaded. The shaded qubits are "site" qubits that represent the sites of the Ising model, and the unshaded qubits are "bond" qubits used to mediate interactions between the site qubits.
- Each site qubit is labeled either (A) or (B), indicating one of two roles a site qubit can play in the GEM protocol (the roles are explained later).
- Each edge is colored using one of six colors, thus partitioning the edges into six groups. This partitioning determines how two-qubit gates can be parallelized, as well as different scheduling patterns that are likely to incur different amounts of error on a noisy quantum processor. Because edges in a group are disjoint, a layer of two-qubit gates can be applied on those edges simultaneously. In fact, it is possible to partition the six colors into three groups of two colors such that the union of each group of two colors is still disjoint. Therefore, only three layers of two-qubit gates are needed to activate every edge. There are 12 ways to so partition the six colors, and each such partition yields a different 3-layer gate schedule.

Now that you have created a plaquette lattice, the next step is to initialize a `GemExperiment` object, passing both the plaquette lattice and the backend that you intend to run the experiment on. The `GemExperiment` class manages the actual implementation of the GEM protocol, including generating circuits, submitting jobs, and analyzing the data. The following code cell initializes the experiment class while restricting the plaquette lattice to only two of the plaquettes (21 qubits), reducing the size of the experiment to ensure that the noise in the hardware doesn't overwhelm the signal.

In [16]:
gem_exp = GemExperiment(plaquette_lattice.filter([9, 12]), backend=backend)

# visualize the plaquette lattice after filtering
plaquette_lattice.filter([9, 12]).draw_qubits()

<Image src="../docs/images/tutorials/nishimori-phase-transition/extracted-outputs/02357c6e-5c83-4ac0-811d-22602d9f33d5-0.avif" alt="Output of the previous code cell" />

De kannst ooch'n Diagramm von de zugrunde liegenden Qubits erzeugen, de das Plaketten-Gitter bilden.

In [12]:
circuits = gem_exp.circuits()
print(f"Total number of circuits: {len(circuits)}")

Total number of circuits: 252


![Output of the previous code cell](../docs/images/tutorials/nishimori-phase-transition/extracted-outputs/a19d63ce-3572-4081-a008-c1332fbbe303-0.avif)

Zusätzlich zu de Qubit-Bezeichnungen un de Kanten, de zeigen, welche Qubits verbunden sin, enthält das Diagramm drei weitere Informationen, de fürs GEM-Protokoll relevant sin:
- Jedes Qubit is entweder schattiert (grau) oder nich schattiert. De schattierten Qubits sin "Site"-Qubits, de de Plätze vom Ising-Modell darstellen, un de nich schattierten Qubits sin "Bond"-Qubits, de verwendet wern, um Interaktionen zwischn de Site-Qubits zu vermitteln.
- Jedes Site-Qubit is entweder (A) oder (B) bezeichnet, was eene von zwei Rollen anzeigt, de'n Site-Qubit im GEM-Protokoll spielen kann (de Rollen wern später erklärt).
- Jede Kante is mit eener von sechs Farben eingefärbt, was de Kanten in sechs Gruppen aufdeilt. Diese Aufdeilung bestimmt, wie Zwei-Qubit-Gatter parallelisiert wern können, sowie verschiedene Zeitpläne, de wohrscheinlich verschiedene Fehlerraten uffm rauschenden Quantenprozessor verursachen. Weil de Kanten inner Gruppe disjunkt sin, kann ne Schicht von Zwei-Qubit-Gattern uff dene Kanten gleichzeitig angelegt wern. Tatsächlich kann mr de sechs Farben in drei Gruppen von zwei Farben aufdeilen, so dass de Vereinigung von jeder Gruppe von zwei Farben immer noch disjunkt is. Deswegen wern nur drei Schichten von Zwei-Qubit-Gattern gebraucht, um jede Kante zu aktivieren. Es jibt 12 Wege, de sechs Farben so uffzudeilen, un jede solche Aufdeilung liefert'n annern 3-Schichten-Gatter-Zeitplan.

Nachdem de'n Plaketten-Gitter erzeugt hast, is der nächste Schritt, 'n `GemExperiment` Objekt zu initialisieren, wo de sowohl das Plaketten-Gitter als ooch das Backend reinjibbst, uff dem de das Experiment durchführen willst. De `GemExperiment` Klasse verwaltet de eigentliche Implementierung vom GEM-Protokoll, was ooch Schaltkreise erzeugt, Jobs einreicht un Daten analysiert. De nächste Code-Zelle initialisiert de Experiment-Klasse un beschränkt das Plaketten-Gitter uff nur zwei von de Plaketten (21 Qubits), um de Größe vom Experiment zu reduzieren un sicherzustellen, dass das Rauschen in der Hardware nich das Signal überwältigt.

In [13]:
# Restrict experiment to the first scheduling pattern
gem_exp.set_experiment_options(schedule_idx=0)

# There are less circuits now
circuits = gem_exp.circuits()
print(f"Total number of circuits: {len(circuits)}")

# Print the RZZ angles swept over
print(f"RZZ angles:\n{gem_exp.parameters()}")

Total number of circuits: 21
RZZ angles:
[0.         0.07853982 0.15707963 0.23561945 0.31415927 0.39269908
 0.4712389  0.54977871 0.62831853 0.70685835 0.78539816 0.86393798
 0.9424778  1.02101761 1.09955743 1.17809725 1.25663706 1.33517688
 1.41371669 1.49225651 1.57079633]


![Output of the previous code cell](../docs/images/tutorials/nishimori-phase-transition/extracted-outputs/02357c6e-5c83-4ac0-811d-22602d9f33d5-0.avif)

'n GEM-Protokoll-Schaltkreis wird mit dene Schritten gebaut:
1. Erzeuge den all-$|+\rangle$ Zustand, indem'n Hadamard-Gatter uff jedes Qubit angelegt wird.
2. Lege'n $R_{ZZ}$ Gatter zwischn jedem Paar von verbundenen Qubits an. Das kann mr mit 3 Schichten von Gattern erreichen. Jedes $R_{ZZ}$ Gatter wirkt uff'n Site-Qubit un'n Bond-Qubit. Wenn das Site-Qubit (B) bezeichnet is, dann is der Winkel fest uff $\frac{\pi}{2}$ gesetzt. Wenn das Site-Qubit (A) bezeichnet is, dann darf der Winkel variieren, was verschiedene Schaltkreise liefert. Standardmäßig is der Bereich von Winkeln uff 21 gleichmäßig verteilte Punkte zwischn $0$ un $\frac{\pi}{2}$, inklusive, gesetzt.
3. Messe jedes Bond-Qubit in der Pauli $X$ Basis. Weil Qubits in der Pauli $Z$ Basis gemessen wern, kann mr das machen, indem'n Hadamard-Gatter vor der Messung angelegt wird.

Beachte, dass das Paper, was in der Einleitung von diesem Tutorial zitiert wird, ne annere Konvention für'n $R_{ZZ}$ Winkel verwendet, de sich durchn Faktor von 2 von der Konvention in diesem Tutorial unterscheidet.

In Schritt 3 wern nur de Bond-Qubits gemessen. Um zu verstehn, in welchem Zustand de Site-Qubits bleiben, is es hilfreich, den Fall zu betrachten, dass der $R_{ZZ}$ Winkel, der in Schritt 2 uff Site-Qubits (A) angelegt wird, gleich $\frac{\pi}{2}$ is. In diesem Fall bleiben de Site-Qubits in'm hochgradig verschränkten Zustand, ähnlich wie beim GHZ-Zustand,

$$
\lvert \text{GHZ} \rangle = \lvert 00 \cdots 00 \rangle + \lvert 11 \cdots 11 \rangle.
$$

Wejen der Zufälligkeit in de Messungserjebnissen könnte der eigentliche Zustand von de Site-Qubits'n annerer Zustand mit Langstreckenordnung sein, zum Beispiel, $\lvert 00110 \rangle + \lvert 11001 \rangle$. Aber der GHZ-Zustand kann wiederhergestellt wern, indem ne Dekodierungsoperation basierend uff de Messungserjebnissen angelegt wird. Wenn der $R_{ZZ}$ Winkel von $\frac{\pi}{2}$ runterjedreht wird, kann de Langstreckenordnung noch immer wiederhergestellt wern bis zu'm kritischen Winkel, der in Abwesenheit von Rauschen ungefähr $0.3 \pi$ is. Unter diesem Winkel zeigt der Zustand, der erzeugt wird, keene Langstreckenverschränkung mehr. Dieser Übergang zwischn der Anwesenheit un Abwesenheit von Langstreckenordnung is der Nishimori Phasenübergang.

In der Beschreibung hier oben wurden de Site-Qubits nich gemessen, un de Dekodierungsoperation kann durchjeführt wern, indem Quantengatter angelegt wern. Im Experiment, wie's in der GEM Suite implementiert is, was diesem Tutorial folgt, wern de Site-Qubits tatsächlich gemessen, un de Dekodierungsoperation wird in'm klassischen Nachbearbeitungsschritt angelegt.

In der Beschreibung hier oben kann de Dekodierungsoperation durchjeführt wern, indem Quantengatter uff de Site-Qubits angelegt wern, um'n Quantenzustand wiederherzustellen. Aber wenn das Ziel is, den Zustand sofort zu messen, zum Beispiel für Charakterisierungszwecke, dann wern de Site-Qubits zusammen mit de Bond-Qubits gemessen, un de Dekodierungsoperation kann in'm klassischen Nachbearbeitungsschritt angelegt wern. So is das Experiment in der GEM Suite implementiert, was diesem Tutorial folgt.

Zusätzlich dazu, dass es vom $R_{ZZ}$ Winkel in Schritt 2 abhängt, der standardmäßig über 21 Werte jeht, hängt der GEM-Protokoll-Schaltkreis ooch vom Zeitplan-Muster ab, was verwendet wird, um de 3 Schichten von $R_{ZZ}$ Gattern zu implementieren. Wie vorher besprochen jibt's 12 solche Zeitplan-Muster. Deswegen is de jesamte Anzahl von Schaltkreisen im Experiment $21 \times 12 = 252$.

De Schaltkreise vom Experiment können mitder `circuits` Methode von der `GemExperiment` Klasse erzeugt wern.

In [14]:
# Get the circuit at index 5
circuit = circuits[5]
# Remove the final measurements to ease visualization
circuit.remove_final_measurements()
# Draw the circuit
circuit.draw("mpl", fold=-1, scale=0.5)

<Image src="../docs/images/tutorials/nishimori-phase-transition/extracted-outputs/fd57d483-c70b-4ad5-b309-15750ad38bac-0.avif" alt="Output of the previous code cell" />

## Step 2: Optimize problem for quantum hardware execution

Transpiling quantum circuits for execution on hardware typically involves a [number of stages](/docs/guides/transpiler-stages). Typically, the stages that incur the most computational overhead are choosing the qubit layout, routing the two-qubit gates to conform to the qubit connectivity of the hardware, and optimizing the circuit to minimize its gate count and depth. In the GEM protocol, the layout and routing stages are unnecessary because the hardware connectivity is already incorporated into the design of the protocol. The circuits already have a qubit layout, and the two-qubit gates are already mapped onto native connections. Furthermore, in order to preserve the structure of the circuit as the $R_{ZZ}$ angle is varied, only very basic circuit optimization should be performed.

The `GemExperiment` class transparently transpiles circuits when executing the experiment. The layout and routing stages are already overridden by default to do nothing, and circuit optimization is performed at a level that only optimizes single-qubit gates. However, you can override or pass additional options using the `set_transpile_options` method. For the sake of visualization, the following code cell manually transpiles the circuit displayed previously, and draws the transpiled circuit.

In [15]:
# Demonstrate setting transpile options
gem_exp.set_transpile_options(
    optimization_level=1  # This is the default optimization level
)
pass_manager = generate_preset_pass_manager(
    backend=backend,
    initial_layout=list(gem_exp.physical_qubits),
    **dict(gem_exp.transpile_options),
)
transpiled = pass_manager.run(circuit)
transpiled.draw("mpl", idle_wires=False, fold=-1, scale=0.5)

<Image src="../docs/images/tutorials/nishimori-phase-transition/extracted-outputs/e9b99d48-8d33-46b5-bff5-480ab1c1c1f2-0.avif" alt="Output of the previous code cell" />

Für de Zwecke von diesem Tutorial is es jenug, nur'n einzelnes Zeitplan-Muster zu betrachten. De nächste Code-Zelle beschränkt das Experiment uffs erste Zeitplan-Muster. Damit hat das Experiment nur 21 Schaltkreise, eener für jeden $R_{ZZ}$ Winkel, über den jejangen wird.

In [10]:
exp_data = gem_exp.run(shots=10_000)

To wait for the results, call the `block_for_results` method of the `ExperimentData` object. This call will cause the interpreter to hang until the jobs are finished.

In [11]:
exp_data.block_for_results()

ExperimentData(GemExperiment, d0d5880a-34c1-4aab-a7b6-c4f58516bc03, job_ids=['cwg12ptmptp00082khhg'], metadata=<5 items>, figure_names=['two_point_correlation.svg', 'normalized_variance.svg', 'plaquette_ops.svg', 'bond_ops.svg'])

De nächste Code-Zelle zeigt'n Diagramm vom Schaltkreis bei Index 5. Um de Größe vom Diagramm zu reduzieren, wern de Messungs-Gatter am Ende vom Schaltkreis entfernt.

In [ ]:
def magnetization_distribution(
    counts_dict: dict[str, int],
) -> dict[str, float]:
    """Compute magnetization distribution from counts dictionary."""
    # Construct dictionary from magnetization to count
    mag_dist = defaultdict(float)
    for bitstring, count in counts_dict.items():
        mag = bitstring.count("0") - bitstring.count("1")
        mag_dist[mag] += count
    # Normalize
    shots = sum(counts_dict.values())
    for mag in mag_dist:
        mag_dist[mag] /= shots
    return mag_dist


# Get counts dictionaries with and without decoding
data = exp_data.data()
# Get the last data point, which is at the angle for the GHZ state
raw_counts = data[-1]["counts"]
# Without decoding
site_indices = [
    i for i, q in enumerate(gem_exp.plaquettes.qubits()) if q.role == "Site"
]
site_raw_counts = defaultdict(int)
for key, val in raw_counts.items():
    site_str = "".join(key[-1 - i] for i in site_indices)
    site_raw_counts[site_str] += val
# With decoding
_, site_decoded_counts = gem_exp.plaquettes.decode_outcomes(
    raw_counts, return_counts=True
)

# Compute magnetization distribution
raw_magnetization = magnetization_distribution(site_raw_counts)
decoded_magnetization = magnetization_distribution(site_decoded_counts)

# Plot
plt.bar(*zip(*raw_magnetization.items()), label="raw")
plt.bar(*zip(*decoded_magnetization.items()), label="decoded", width=0.3)
plt.legend()
plt.xlabel("Magnetization")
plt.ylabel("Frequency")
plt.title("Magnetization distribution with and without decoding")

Text(0.5, 1.0, 'Magnetization distribution with and without decoding')

<Image src="../docs/images/tutorials/nishimori-phase-transition/extracted-outputs/8ead3582-16df-4616-836c-bdce867ad6b8-1.avif" alt="Output of the previous code cell" />

![Output of the previous code cell](../docs/images/tutorials/nishimori-phase-transition/extracted-outputs/fd57d483-c70b-4ad5-b309-15750ad38bac-0.avif)

## Schritt 2: Problem für Quanten-Hardware-Ausführung optimieren
Das Transpilieren von Quantenschaltkreisen für de Ausführung uff Hardware umfasst typischerweise [ne ganze Reihe von Etappen](/guides/transpiler-stages). Typischerweise sin de Etappen, de am meisten rechenintensiv sin, de Auswahl vom Qubit-Layout, das Routing von de Zwei-Qubit-Gattern, um sich an de Qubit-Konnektivität von der Hardware anzupassen, un de Optimierung vom Schaltkreis, um seine Gatter-Anzahl un Tiefe zu minimieren. Im GEM-Protokoll sin de Layout- un Routing-Etappen unnötig, weil de Hardware-Konnektivität schon ins Design vom Protokoll eingebaut is. De Schaltkreise ham schon'n Qubit-Layout, un de Zwei-Qubit-Gatter sin schon uff native Verbindungen abgebildet. Darüber hinaus sollte nur sehr einfache Schaltkreis-Optimierung durchjeführt wern, um de Struktur vom Schaltkreis zu erhalten, wenn der $R_{ZZ}$ Winkel variiert wird.

De `GemExperiment` Klasse transpiliert Schaltkreise transparent, wenn das Experiment durchjeführt wird. De Layout- un Routing-Etappen sin standardmäßig schon überschrieben, um nix zu tun, un Schaltkreis-Optimierung wird uffm Niveau durchjeführt, was nur Ein-Qubit-Gatter optimiert. Aber de kannst zusätzliche Optionen überschreiben oder anjeben, indem de de `set_transpile_options` Methode verwendest. Für de Zwecke von der Visualisierung transpiliert de nächste Code-Zelle manuell den Schaltkreis, der vorher anjezeigt wurde, un zeigt den transpilierten Schaltkreis.

In [13]:
exp_data.figure("two_point_correlation")

<Image src="../docs/images/tutorials/nishimori-phase-transition/extracted-outputs/4ecb25c8-e572-49af-a879-9943039db131-0.avif" alt="Output of the previous code cell" />

![Output of the previous code cell](../docs/images/tutorials/nishimori-phase-transition/extracted-outputs/e9b99d48-8d33-46b5-bff5-480ab1c1c1f2-0.avif)

## Schritt 3: Mit Qiskit Primitives ausführen
Um de GEM-Protokoll-Schaltkreise uff der Hardware auszuführen, ruf de `run` Methode vom `GemExperiment` Objekt uff. De kannst de Anzahl von Shots anjeben, de de aus jedem Schaltkreis sampeln willst. De `run` Methode liefert'n [ExperimentData](https://qiskit-community.github.io/qiskit-experiments/stubs/qiskit_experiments.framework.ExperimentData.html) Objekt, was de inner Variable speichern solltest. Beachte, dass de `run` Methode nur Jobs einreicht ohne uff se zu warten bis se fertig sin, so dass's'n nich-blockierender Aufruf is.

In [14]:
exp_data.figure("normalized_variance")

<Image src="../docs/images/tutorials/nishimori-phase-transition/extracted-outputs/2b351d68-3924-445a-94ef-047b16214e8a-0.avif" alt="Output of the previous code cell" />

Um uff de Erjebnisse zu warten, ruf de `block_for_results` Methode vom `ExperimentData` Objekt uff. Dieser Aufruf lässt den Interpreter hängen, bis de Jobs fertig sin.

In [15]:
gem_exp = GemExperiment(
    plaquette_lattice.filter(range(3, 9)), backend=backend
)
gem_exp.set_experiment_options(schedule_idx=0)
exp_data = gem_exp.run(shots=10_000)
exp_data.block_for_results()
exp_data.figure("normalized_variance")

<Image src="../docs/images/tutorials/nishimori-phase-transition/extracted-outputs/08581c09-a6a5-4a56-9fc4-abf22b063c6a-0.avif" alt="Output of the previous code cell" />

In [16]:
gem_exp = GemExperiment(plaquette_lattice, backend=backend)
gem_exp.set_experiment_options(schedule_idx=0)
exp_data = gem_exp.run(shots=10_000)
exp_data.block_for_results()
exp_data.figure("normalized_variance")

<Image src="../docs/images/tutorials/nishimori-phase-transition/extracted-outputs/37e9a4cd-6efb-4ade-ad09-8139db9d58e9-0.avif" alt="Output of the previous code cell" />

## Schritt 4: Nachbearbeiten un Erjebniss im jewünschten klassischen Format zurückjeben
Beim $R_{ZZ}$ Winkel von $\frac{\pi}{2}$ würde der dekodierte Zustand der GHZ-Zustand in Abwesenheit von Rauschen sein. De Langstreckenordnung vom GHZ-Zustand kann visualisiert wern, indem de Magnetisierung von de jemessenen Bitstrings abgebildet wird. De Magnetisierung $M$ wird definiert als de Summe von de Ein-Qubit-Pauli $Z$ Operatoren,
$$
M = \sum_{j=1}^N Z_j,
$$
wo $N$ de Anzahl von Site-Qubits is. Sein Wert für'n Bitstring is gleich dem Unterschied zwischn der Anzahl von Nullen un der Anzahl von Einsen. Das Messen vom GHZ-Zustand liefert den all-Null Zustand oder den all-Eins Zustand mit jleicher Wahrscheinlichkeit, so dass de Magnetisierung $+N$ de halbe Zeit un $-N$ de annere halbe Zeit wäre. In der Anwesenheit von Fehlern wejen Rauschen würden ooch annere Werte ufftreten, aber wenn das Rauschen nich zu groß is, würde de Verteilung noch immer Spitzen bei $+N$ un $-N$ zeigen.

Für de rohen Bitstrings vor der Dekodierung würde de Verteilung von der Magnetisierung gleichbedeutend mit denen von jleichmäßig zufälligen Bitstrings sein, in Abwesenheit von Rauschen.

De nächste Code-Zelle zeigt de Magnetisierung von de rohen Bitstrings un de dekodierten Bitstrings beim $R_{ZZ}$ Winkel von $\frac{\pi}{2}$.